<h1> Fashion MNIST Mini Project Summary

In [ ]:
# Loading in modules
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix


Comparison of the two best models
- Simple cnn with one conv layer and 30 filters
- Deeper cnn with 4 conv layers and 240 filters in the first layer halved at each subsequent layer
